In [103]:
import numpy as np
import plotly.graph_objects as go
import torch
from jaxtyping import Float
from torch import Tensor
from typing import Union, Tuple
from nerfstudio.data.scene_box import SceneBox, OrientedBox

In [108]:
""" nsa_3d_ns = np.array([[-0.078125 , -0.1875   , -0.8261623], 
                    [-0.078125 , -0.078125 , -0.8255522], 
                    [ 0.046875 , -0.1875   , -0.82176167],
                    [ 0.046875 , -0.078125 , -0.8211517]]) """
nsa_3d_ns = np.array([[-0.078125 , -0.1875   , -0.7561623], 
                    [-0.078125 , -0.078125 , -0.8255522], 
                    [ 0.046875 , -0.1875   , -0.73176167],
                    [ 0.046875 , -0.078125 , -0.8211517]])
# Define a constant vector that only shifts the z coordinate
#z_offset = np.array([0, 0, 0.3])



# self.object_aabb derived from occupancy grid: tensor([[-0.0781, -0.1875, -0.8750], [ 0.0469, -0.0781, -0.5781]], device='cuda:0')
# x_min, y_min, z_min = self.object_aabb[0]
# x_max, y_max, z_max = self.object_aabb[1]

object_aabb = np.array([[-0.0781, -0.1875, -0.8750], [ 0.0469, -0.0781, -0.5781]])
# plot aabb
min_point = object_aabb[0]
max_point = object_aabb[1]
# get the vertices of the aabb, in the order of     
    # edges = {
    #     "x": [(0, 4), (1, 5), (2, 6), (3, 7)],
    #     "y": [(0, 2), (1, 3), (4, 6), (5, 7)],
    #     "z": [(0, 1), (2, 3), (4, 5), (6, 7)]
    # }
vertices = np.array([[min_point[0], min_point[1], min_point[2]],
                         [min_point[0], min_point[1], max_point[2]],
                         [min_point[0], max_point[1], min_point[2]],
                         [min_point[0], max_point[1], max_point[2]],
                         [max_point[0], min_point[1], min_point[2]],
                         [max_point[0], min_point[1], max_point[2]],
                         [max_point[0], max_point[1], min_point[2]],
                         [max_point[0], max_point[1], max_point[2]]])
# find lower four points of vertices
#lower_vertices = vertices[:4, :]
lower_vertices = np.vstack((vertices[:1, :], vertices[2:3, :], vertices[4:5, :], vertices[6:7, :]))

# find upper four points of vertices
#upper_vertices = vertices[4:, :]
upper_vertices = np.vstack((vertices[1:2, :], vertices[3:4, :], vertices[5:6, :], vertices[7:8, :]))

# find max z-coordinate among the lower four points of vertices
max_z_lower_vertices = np.max(lower_vertices[:, 2])
# find min z-coordinate among the nsa_3d_ns
min_z_nsa_3d_ns = np.min(nsa_3d_ns[:, 2])

# find z axis distance between lower and min z-coordinate among the nsa_3d_ns
z_offset = max_z_lower_vertices - min_z_nsa_3d_ns

parallel_points = nsa_3d_ns + np.array([0, 0, z_offset])

# Find the maximum z-coordinate among the upper four points of vertices
max_z_aabb = np.max(vertices[4:, 2])
min_z_parallel_points = np.min(parallel_points[:, 2])
z_shift = max_z_aabb - min_z_parallel_points

upper_obb_vertices = parallel_points + np.array([0, 0, z_shift])
#z_offset = np.array([0, 0, z_shift])

# obb = OrientedBox(R=R, T=T, S=S)
'''
R: Float[Tensor, "3 3"]
    """R: rotation matrix."""
    T: Float[Tensor, "3"]
    """T: translation vector."""
    S: Float[Tensor, "3"]
'''	
# plot max_z_lower_vertices
# plot vertices

obb_vertices = np.vstack((nsa_3d_ns, upper_obb_vertices))

fig = go.Figure(data=[
    go.Scatter3d(x=vertices[:,0], y=vertices[:,1], z=vertices[:,2], mode='markers', marker=dict(size=5, color='red'), name='aabb_vertices'),
    #go.Scatter3d(x=nsa_3d_ns[:,0], y=nsa_3d_ns[:,1], z=nsa_3d_ns[:,2], mode='markers', marker=dict(size=5, color='blue'), name='nsa_intersections(dummy)'),
    #go.Scatter3d(x=upper_obb_vertices[:,0], y=upper_obb_vertices[:,1], z=upper_obb_vertices[:,2], mode='markers', marker=dict(size=5, color='yellow'), name='upper_obb_vertices')
    go.Scatter3d(x=obb_vertices[:,0], y=obb_vertices[:,1], z=obb_vertices[:,2], mode='markers', marker=dict(size=5, color='blue'), name='obb_vertices(dummy)')
])

fig.update_layout(
    scene=dict(
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=1)  
    )
)

# combine vertices and nsa_3d_ns
#obb_vertices = np.vstack((vertices, upper_obb_vertices))

fig.show()


In [105]:
# # refined oriented box
        # # another way to get R (Rodrigues)
        # target_z = scaled_transformed_vertices[1] - scaled_transformed_vertices[0]
        # normalized_target_z = target_z / torch.linalg.norm(target_z)
        # source_z = torch.tensor([0., 0., 1.])
        # # print(source_z, normalized_target_z)
        # v = torch.cross(source_z, normalized_target_z)
        # c = torch.dot(source_z, normalized_target_z)
        # s = v.norm()
        # kmat = torch.tensor([[0, -v[2], v[1]],
        #                     [v[2], 0, -v[0]],
        #                     [-v[1], v[0], 0]])
        # obb_R = torch.eye(3) + kmat + kmat @ kmat * ((1 - c) / (s ** 2)) #obb_R.shape

        # # get S
        # target_x = scaled_transformed_vertices[4] - scaled_transformed_vertices[0]
        # target_y = scaled_transformed_vertices[2] - scaled_transformed_vertices[0]
        # obb_S = torch.tensor([torch.linalg.norm(target_x), torch.linalg.norm(target_y), torch.linalg.norm(target_z)])

        # # get T
        # obb_T = torch.mean(scaled_transformed_vertices, dim=0)

        # self.object_obb = OrientedBox(R=obb_R, T=obb_T, S=obb_S)


In [106]:
target_z = vertices[1] - vertices[0]
#          blue point - yellow point                                
normalized_target_z = target_z / np.linalg.norm(target_z)
# plot target_z
# plot vertices
# plot vertices[1]
# plot vertices[0]
fig = go.Figure(data=[
    go.Scatter3d(x=vertices[:,0], y=vertices[:,1], z=vertices[:,2], mode='markers', marker=dict(size=5, color='red'), name='aabb_vertices'),
    go.Scatter3d(x=[vertices[1,0]], y=[vertices[1,1]], z=[vertices[1,2]], mode='markers', marker=dict(size=5, color='blue'), name='vertices[1]'),
    go.Scatter3d(x=[vertices[0,0]], y=[vertices[0,1]], z=[vertices[0,2]], mode='markers', marker=dict(size=5, color='yellow'), name='vertices[0]'),
    

    ])

fig.update_layout(
    scene=dict(
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=1)  
    )
)
fig.show()

In [107]:
# refined oriented box
# combine nsa_3d_ns and upper_obb_vertices
obb_vertices = np.vstack((nsa_3d_ns, upper_obb_vertices))


'''
[[-0.0781     -0.1875     -0.875     ]
 [-0.0781     -0.1875     -0.5781    ]
 [-0.0781     -0.0781     -0.875     ]
 [-0.0781     -0.0781     -0.5781    ]
 [ 0.0469     -0.1875     -0.875     ]
 [ 0.0469     -0.1875     -0.5781    ]
 [ 0.0469     -0.0781     -0.875     ]
 [ 0.0469     -0.0781     -0.5781    ]
 [-0.078125   -0.1875     -0.5087101 ]
 [-0.078125   -0.078125   -0.5781    ]
 [ 0.046875   -0.1875     -0.48430947]
 [ 0.046875   -0.078125   -0.5736995 ]]
'''
# plot obb_vertices
# plot obb_vertices[1]
# plot obb_vertices[0]

# # refined oriented box
        # # another way to get R (Rodrigues)
        # target_z = scaled_transformed_vertices[1] - scaled_transformed_vertices[0]
        # normalized_target_z = target_z / torch.linalg.norm(target_z)
        # source_z = torch.tensor([0., 0., 1.])
        # # print(source_z, normalized_target_z)
        # v = torch.cross(source_z, normalized_target_z)
        # c = torch.dot(source_z, normalized_target_z)
        # s = v.norm()
        # kmat = torch.tensor([[0, -v[2], v[1]],
        #                     [v[2], 0, -v[0]],
        #                     [-v[1], v[0], 0]])
        # obb_R = torch.eye(3) + kmat + kmat @ kmat * ((1 - c) / (s ** 2)) #obb_R.shape

        # # get S
        # target_x = scaled_transformed_vertices[4] - scaled_transformed_vertices[0]
        # target_y = scaled_transformed_vertices[2] - scaled_transformed_vertices[0]
        # obb_S = torch.tensor([torch.linalg.norm(target_x), torch.linalg.norm(target_y), torch.linalg.norm(target_z)])

        # # get T
        # obb_T = torch.mean(scaled_transformed_vertices, dim=0)

        # self.object_obb = OrientedBox(R=obb_R, T=obb_T, S=obb_S)
#print(obb_vertices[1])
# 1(low) - 5(high)
# verctor 1 -> 5
target_z = obb_vertices[5] - obb_vertices[1]
normalized_target_z = target_z / np.linalg.norm(target_z)
base_z = np.array([0., 0., 1.])
v = np.cross(base_z, normalized_target_z)
c = np.dot(base_z, normalized_target_z)
s = np.linalg.norm(v)
kmat = np.array([[0, -v[2], v[1]],
                [v[2], 0, -v[0]],
                [-v[1], v[0], 0]])
obb_R = np.eye(3) + kmat + kmat @ kmat * ((1 - c) / (s ** 2))
# get S
target_x = obb_vertices[2] - obb_vertices[0]
target_y = obb_vertices[3] - obb_vertices[2]
obb_S = np.array([np.linalg.norm(target_x), np.linalg.norm(target_y), np.linalg.norm(target_z)])
# get T
obb_T = np.mean(obb_vertices, axis=0)
# plot obb_vertices
object_obb = OrientedBox(R=obb_R, T=obb_T, S=obb_S)
idx_start = 2
idx_end = 3
fig = go.Figure(data=[
    go.Scatter3d(x=obb_vertices[:,0], y=obb_vertices[:,1], z=obb_vertices[:,2], mode='markers', marker=dict(size=5, color='red'), name='obb_vertices'),
    go.Scatter3d(x=[obb_vertices[idx_start,0]], y=[obb_vertices[idx_start,1]], z=[obb_vertices[idx_start,2]], mode='markers', marker=dict(size=5, color='yellow'), name='start'),
    go.Scatter3d(x=[obb_vertices[idx_end,0]], y=[obb_vertices[idx_end,1]], z=[obb_vertices[idx_end,2]], mode='markers', marker=dict(size=5, color='blue'), name='end')
])
# plot [-0.078125  -0.078125  -0.8255522]
fig.update_layout(
    scene=dict(
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=1)  
    )
)
fig.show()
# start: yellow point

C:\Users\10744\AppData\Local\Temp\ipykernel_2288\213163700.py:59: RuntimeWarning:

invalid value encountered in scalar divide

